In [1]:
# !pip install rank_bm25
# !pip install mecab-python3

import pandas as pd
from tqdm import tqdm
from ast import literal_eval
import MeCab
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.document_loaders import DataFrameLoader
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

## Vector Store 생성 
- Dense Retrieval 관련

### 데이터 준비

In [3]:
# 한국사 데이터 대한 임베딩 생성
book = pd.read_csv("../data/books/korean_history_textbook.csv")
term = pd.read_csv("../data/books/korean_history_term.csv")

# 25자 이상인 문서만 남기기
book = book[book.context.str.len() >= 25]

# 결합하기
df = pd.concat([book, term], axis=0, ignore_index=True)

In [ ]:
loader = DataFrameLoader(df, page_content_column='context')
documents = loader.load()

# text split (chunking)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator=". ",
    chunk_size=500,
    chunk_overlap=100,
    encoding_name='cl100k_base'
)
split_docs = text_splitter.split_documents(tqdm(documents))
len(split_docs)

100%|██████████| 5302/5302 [00:00<00:00, 753250.00it/s]


5374

# 임베딩 생성

### Sparse Embedding

In [4]:
# MeCab 형태소 분석기 생성
mecab = MeCab.Tagger()

# 명사 추출 함수 정의
def extract_nouns(text):
    try:
        parsed = mecab.parse(text)  # 텍스트 분석
        nouns = []
        for line in parsed.splitlines():
            if '\t' in line:  # 형태소 분석 결과가 탭으로 구분됨
                word, feature = line.split('\t')
                # 품사 정보에서 명사인지 확인
                if feature.startswith('NN'):  # 'NN'으로 시작하는 경우 명사
                    nouns.append(word)
        return ' '.join(nouns)  # 명사 리스트를 띄어쓰기로 연결
    except Exception as e:
        print(f"Error processing text: {text}, Error: {e}")
        return text  # 처리 실패 시 원본 반환

In [5]:
# split_docs에 대해 명사 추출
split_docs_nouns = [
    Document(
        metadata={
            "section": row.metadata['section'],
            "title": row.metadata['title'],
            "original": row.page_content
            },
        page_content=extract_nouns(row.page_content)
    )
    for row in split_docs
]

In [6]:
topk = 5

# BM25Retriever 생성
bm25_retriever = BM25Retriever.from_documents(split_docs_nouns)
bm25_retriever.k = topk  # BM25 검색기의 k 설정 (가장 유사한 문서 5개 반환)

### Dense Embedding

In [7]:
# dense embedding
model_name = 'dragonkue/BGE-m3-ko'
device = 'cuda'

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True},
)

# # 한번에 split_docs를 다 저장하면 cuda out of memory 에러가 발생해서, 한 문서씩 저장되도록 코드 작성해둔 상태입니다
# vector_store = FAISS.from_documents(
#     [split_docs[0]],
#     embedding=embeddings,
#     distance_strategy=DistanceStrategy.COSINE
# )

# with tqdm(total=len(split_docs[1:]), desc="Ingesting documents") as pbar:
#     for idx, doc in enumerate(split_docs[1:]):
#         vector_store.add_documents([doc])
#         pbar.update(1)

# # DB 저장
# vector_store.save_local("../db/faiss_dragonkue_BGE-m3-ko_korean_history_600_200_split")

# # DB에 저장된 문서 개수 확인
# doc_count = vector_store.index.ntotal
# doc_count

# vector store가 저장되어 있다면 해당 DB 불러오기
persist_path = "../db/faiss_dragonkue_BGE-m3-ko_korean_history_600_200_split"
vector_store = FAISS.load_local(
            persist_path,
            embeddings=embeddings,
            distance_strategy=DistanceStrategy.COSINE,
            allow_dangerous_deserialization=True,
        )

In [8]:
# dense retriever 지정
faiss_retriever = vector_store.as_retriever(search_kwargs={"k":topk})

# Retrieval 성능 확인
- trainset 중 한국사 문제에 대한 retrieval 성능 확인

### retrieval 성능 검증용 평가셋

In [9]:
# 키워드 포함된 데이터 (각 문제별로 2개 키워드 포함)
eval_set = pd.read_csv("../data/rag_eval_kor_history.csv")

# 프롬프트 현식지정
prompt = "{paragraph}\n{question}\n{choices}"

# input 형태 맞추기
eval_set['user_message'] = ""
for idx, row in eval_set.iterrows():
    problems = literal_eval(row['problems'])
    question = problems['question']
    choices = problems['choices']
    choices_str = " ".join(choices)
    user_message = prompt.format(paragraph=row['paragraph'], question=question, choices=choices_str)
    eval_set.loc[idx, 'user_message'] = user_message

eval_set.head()

,id,paragraph,problems,keyword,user_message
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",{'question': '상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두...,"남인, 기사환국","상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)..."
1,generation-for-nlp-426,"한양이라는 이름은 조선 왕조가 건국되면서 수도로 삼은 지역의 이름으로, 한강의 북쪽...","{'question': '한양에 대한 설명으로 옳지 않은 것은?', 'choices...","남경, 한양","한양이라는 이름은 조선 왕조가 건국되면서 수도로 삼은 지역의 이름으로, 한강의 북쪽..."
2,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,"{'question': '(가) 지역에 대한 설명으로 옳은 것은?', 'choice...","서경, 동녕부",나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...
3,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,"{'question': '밑줄 친 ‘그’에 대한 설명으로 옳은 것은?', 'choi...","김유신, 김춘추",이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....
4,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...","{'question': '(가) 인물이 추진한 정책으로 옳지 않은 것은?', 'ch...","흥선대원군, 비변사","선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가..."


### Retrieve
- retrieved document 저장 (metadata 제외하고 본문(page_content)만 저장)

In [10]:
# sparse retrieval
sparse_eval = eval_set.copy()
sparse_eval['reference'] = ""
for idx, row in sparse_eval.iterrows():
    query = row['user_message']
    processed_query = extract_nouns(query)
    references = [ref.metadata['original'] for ref in bm25_retriever.invoke(processed_query)]
    sparse_eval.loc[idx, 'reference'] = str(references)

sparse_eval.head()

,id,paragraph,problems,keyword,user_message,reference
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",{'question': '상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두...,"남인, 기사환국","상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",['기해예송 때는 『경국대전(經國大典)』에서 장자와 차자 모두 기년복을 입는다는 규...
1,generation-for-nlp-426,"한양이라는 이름은 조선 왕조가 건국되면서 수도로 삼은 지역의 이름으로, 한강의 북쪽...","{'question': '한양에 대한 설명으로 옳지 않은 것은?', 'choices...","남경, 한양","한양이라는 이름은 조선 왕조가 건국되면서 수도로 삼은 지역의 이름으로, 한강의 북쪽...",['조선 후기 시전(市廛)은 도성 상업의 중심지였다. 그러나 18세기 후반 종로에 ...
2,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,"{'question': '(가) 지역에 대한 설명으로 옳은 것은?', 'choice...","서경, 동녕부",나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,['몽골 침략으로 소실된 초조대장경을 대신하여고종때에는 대장경을 다시 만들었다. 대...
3,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,"{'question': '밑줄 친 ‘그’에 대한 설명으로 옳은 것은?', 'choi...","김유신, 김춘추",이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,"['안원왕(安原王, 재위 531~545)이 죽은 뒤 왕위 계승 분쟁으로 고구려의 내..."
4,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...","{'question': '(가) 인물이 추진한 정책으로 옳지 않은 것은?', 'ch...","흥선대원군, 비변사","선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",['고종의 즉위(1863)로 정치적 실권을 잡은흥선 대원군은 왕조의 위기를 극복하고...


In [11]:
# dense retrieval
dense_eval = eval_set.copy()
dense_eval['reference'] = ""
for idx, row in dense_eval.iterrows():
    query = row['user_message']
    references = [ref.page_content for ref in faiss_retriever.invoke(query)]
    dense_eval.loc[idx, 'reference'] = str(references)

dense_eval.head()

,id,paragraph,problems,keyword,user_message,reference
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",{'question': '상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두...,"남인, 기사환국","상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...","['장자의 며느리가 죽었을 경우에는 기년복을, 차자의 며느리가 죽었을 때는 9개월만..."
1,generation-for-nlp-426,"한양이라는 이름은 조선 왕조가 건국되면서 수도로 삼은 지역의 이름으로, 한강의 북쪽...","{'question': '한양에 대한 설명으로 옳지 않은 것은?', 'choices...","남경, 한양","한양이라는 이름은 조선 왕조가 건국되면서 수도로 삼은 지역의 이름으로, 한강의 북쪽...","['남경은 1308년(충렬왕 34) 한양부(漢陽府)로 격하되고, 윤(尹)⋅판관(判官..."
2,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,"{'question': '(가) 지역에 대한 설명으로 옳은 것은?', 'choice...","서경, 동녕부",나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,['대장경판을 새긴 것은 부처의 힘으로 몽골군의 침입을 물리치고자 하는 염원에서 비...
3,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,"{'question': '밑줄 친 ‘그’에 대한 설명으로 옳은 것은?', 'choi...","김유신, 김춘추",이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,['대조영의 뒤를 이은무왕때에는 영토 확장에 힘을 기울여 동북방의 여러 세력을 복속...
4,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...","{'question': '(가) 인물이 추진한 정책으로 옳지 않은 것은?', 'ch...","흥선대원군, 비변사","선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",['공민왕의 반원 자주 정책은기철로 대표되던 친원 세력을 숙청하는 데서부터 시작하였...


# Retrieve 성능 평가
- '관련 문서' 정의
    - 두 키워드(`keyword` 열) 중 하나라도 포함하고 있는 문서 (`reference` 열)
- 평가 metric
    - Hit@K: K개 문서 중 관련 문서가 하나라도 존재하는지에 대한 metric
    - MRR@K: K개 문서 중 관련 문서가 얼마나 상위에 존재하는지에 대한 metric
- 띄어쓰기 무시
    - 키워드의 띄어쓰기에 따른 성능 차이 방지해야 함 (예: "흥선대원군"이 키워드인데, 문서에 "흥선 대원군"이라고 나와있어 연관 문서가 아니라고 판단되는 경우)
    - 띄어쓰기에 상관없이 평가될 수 있도록 문서의 공백을 제거한 뒤 매칭 수행함

In [12]:
# hit@K, MRR@K 계산하는 함수
def evaluate_hit_mrr(df):
    result_df = df.copy()
    total_hits = 0  # 전체 히트 수
    total_reciprocal_rank = 0.0  # 전체 역순위 합계
    total_rows = len(df)  # 전체 데이터 수
    result_df[['hit', "rank"]] = [False,0]

    for idx, row in df.iterrows():
        # 키워드를 쉼표로 분리
        keywords = [kw for kw in row['keyword'].split(',')]

        # 검색된 문서 리스트
        references = eval(row['reference'])

        rank = 0  # 초기 순위
        for i, doc in enumerate(references):
            # 문서의 공백 제거
            doc_no_space = doc.replace(' ', '').replace('\n', '')
            found = False
            for kw in keywords:
                # 키워드의 공백 제거
                kw_no_space = kw.replace(' ', '')
                # 키워드가 문서에 포함되어 있는지 확인
                if kw_no_space in doc_no_space:
                    rank = i + 1  # 순위는 1부터 시작
                    found = True
                    break  # 키워드를 찾았으므로 내부 루프 종료
            if found:
                break  # 매칭된 문서를 찾았으므로 외부 루프 종료

        if rank > 0:
            result_df.loc[idx, 'hit'] = True
            result_df.loc[idx, 'rank'] = rank
            total_hits += 1  # 히트 증가
            reciprocal_rank = 1.0 / rank  # 역순위 계산
            total_reciprocal_rank += reciprocal_rank  # 역순위 합계에 추가
        else:
            pass  # 매칭된 문서가 없는 경우 처리 없음

    # Hit@K와 MRR@K 계산
    hit_at_k = total_hits / total_rows
    mrr_at_k = total_reciprocal_rank / total_rows

    return result_df, hit_at_k, mrr_at_k

In [13]:
# pd.set_option("max_colwidth", None)

# 결과 계산 및 출력
sparse_eval_res, hit_at_k_sparse, mrr_at_k_sparse = evaluate_hit_mrr(sparse_eval)
dense_eval_res, hit_at_k_dense, mrr_at_k_dense = evaluate_hit_mrr(dense_eval)

print("[Sparse Retrieval]")
print(f"Hit@{topk}: {hit_at_k_sparse:.4f}")
print(f"MRR@{topk}: {mrr_at_k_sparse:.4f}")
print("-"*100)
print("[Dense Retrieval]")
print(f"Hit@{topk}: {hit_at_k_dense:.4f}")
print(f"MRR@{topk}: {mrr_at_k_dense:.4f}")
print("-"*100)

# sparse_eval_res.head(2)
# dense_eval_res.head(2)

[Sparse Retrieval]
Hit@5: 0.7123
MRR@5: 0.5146
----------------------------------------------------------------------------------------------------
[Dense Retrieval]
Hit@5: 0.7671
MRR@5: 0.5728
----------------------------------------------------------------------------------------------------


### 각 문제에 대한 결과 하나씩 조회

In [14]:
# sparse retrieval
idx = 0
row = sparse_eval_res.iloc[idx]
references = eval(row['reference'])

print(f"[결과]")
print(f"Hit: {row['hit']}")    # 검색된 문서 중, 연관 문서가 존재하는지 여부 (T/F)
print(f"Rank: {row['rank']}\n")  # 검색된 문서 중, 연관 문서의 순위 (1, ..., TopK)
print(f"[키워드]\n{row['keyword']}\n")
print(f"[문제]\n{row['user_message']}\n")
print("[검색 결과]")
for i, ref in enumerate(references):
    print(f"Result {i+1}: {ref}")

[결과]
Hit: True
Rank: 2

[키워드]
남인, 기사환국

[문제]
상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服) 절차에 대하여 논한 것이 신과는 큰 차이가 있었습니다 . 장자를 위하여 3년을 입는 까닭은 위로 ‘정체(正體)’가 되기 때문이고 또 전 중(傳重: 조상의 제사나 가문의 법통을 전함)하기 때문입니다 . …(중략) … 무엇보다 중요한 것은 할아버지와 아버지의 뒤를 이은 ‘정체’이지, 꼭 첫째이기 때문에 참 최 3년 복을 입는 것은 아닙니다 .”라고 하였다 .－현종실록 －ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 .ㄷ.정조 시기에 탕평 정치의 한 축을 이루었다 .ㄹ.이 이와 성혼의 문인을 중심으로 형성되었다.
상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?
ㄱ, ㄴ ㄱ, ㄷ ㄴ, ㄹ ㄷ, ㄹ

[검색 결과]
Result 1: 기해예송 때는 『경국대전(經國大典)』에서 장자와 차자 모두 기년복을 입는다는 규정에 따라 장렬왕후는 1년 동안 상복(喪服)을 입었다. 표면적으로는 서인이 승리하였으나 효종의 지위 문제가 완전히 마무리되지 못한 상황이었기 때문에 언제든지 문제가 다시 불거질 수 있었다. 결국 1674년(현종 15) 효종 비 인선왕후의 죽음으로 또다시 장렬왕후가 상복을 입어야 하는 상황에서 복상 기간을 둘러싼 2차 논쟁, 갑인예송이 벌어졌다.
Result 2: 박세채의 탕평론을 수용한 것은 숙종에 뒤이어 즉위한 영조에 의해서이다. 영조(英祖, 재위 1724~1776)는 즉위 초부터 ‘탕평’을 표방했다. 그러나 집권 초반에는 과거의 관행이 이어지면서 소론 정권에서 노론 정권으로, 또다시 소론 정권으로 바뀌는 환국 정치가 계속되었다. 1728년(영조 4) 소론⋅남인 급진파에 의한 전국 규모의 반란인 무신란(戊申亂)이 발생하자, 이를 수습한 영조는 붕당 타파를 전면에 내세운 기유대처분(己酉大處分)을 내림으로써 탕평책을 통한 정치를 본격적으로 시작했다.

In [15]:
# dense retrieval
idx = 0
row = dense_eval_res.iloc[idx]
references = eval(row['reference'])

print(f"[결과]")
print(f"Hit: {row['hit']}")    # 검색된 문서 중, 연관 문서가 존재하는지 여부 (T/F)
print(f"Rank: {row['rank']}\n")  # 검색된 문서 중, 연관 문서의 순위 (1, ..., TopK)
print(f"[키워드]\n{row['keyword']}\n")
print(f"[문제]\n{row['user_message']}\n")
print("[검색 결과]")
for i, ref in enumerate(references):
    print(f"Result {i+1}: {ref}")

[결과]
Hit: True
Rank: 1

[키워드]
남인, 기사환국

[문제]
상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服) 절차에 대하여 논한 것이 신과는 큰 차이가 있었습니다 . 장자를 위하여 3년을 입는 까닭은 위로 ‘정체(正體)’가 되기 때문이고 또 전 중(傳重: 조상의 제사나 가문의 법통을 전함)하기 때문입니다 . …(중략) … 무엇보다 중요한 것은 할아버지와 아버지의 뒤를 이은 ‘정체’이지, 꼭 첫째이기 때문에 참 최 3년 복을 입는 것은 아닙니다 .”라고 하였다 .－현종실록 －ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 .ㄷ.정조 시기에 탕평 정치의 한 축을 이루었다 .ㄹ.이 이와 성혼의 문인을 중심으로 형성되었다.
상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?
ㄱ, ㄴ ㄱ, ㄷ ㄴ, ㄹ ㄷ, ㄹ

[검색 결과]
Result 1: 장자의 며느리가 죽었을 경우에는 기년복을, 차자의 며느리가 죽었을 때는 9개월만 상복을 입는 대공복(大功服)을 입어야 하는 상황에서 또다시 복상 기간을 둘러싼 논쟁이 벌어졌다. 결국 현종은 효종을 차자로 보아서는 안 된다고 공표하면서 남인의 손을 들어주었다. 이는 숙종 즉위 초 남인 정권이 들어서는 배경이 되기도 했다.
Result 2: 탕평 정치는영조때 자리잡았다.영조는 왕과 신하 사이의 의리를 바로 세워야 한다며,붕당을 없애자는 논리에 동의하는탕평파를 중심으로 정국을 운영하였다. 그리고붕당의 뿌리를 제거하기 위하여공론의 주재자로서 인식되던 산림의 존재를 인정하지 않았고, 그들의 본거지인서원을 대폭 정리하였다. 아울러이조 전랑의 권한을 약화시키기 위하여 그들이 자신의 후임자를 천거하고,3사의 관리를 선발할 수 있게 해 주던 관행을 없앴다. 그러나이조 전랑의 후임자 천거권은 이후정조대에 가서야 완전히 폐지되었다.
Result 3: 붕당이 형성된 직후에는 서인은 동인에 비하여 열세였으나 1589년(선조 22)에 일어난 정여립의

---

# Reranking
- bm25에서는 형태소 분석기로 추출된 명사를, reranking에서는 원문을 이용하도록 클래스 수정
    - `OriginalCrossEncoderReranker` 클래스
    - `OriginalContextualCompressionRetriever` 클래스
- 참고
    - https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/retrievers/document_compressors/cross_encoder_rerank.py
    - https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/retrievers/contextual_compression.py


In [16]:
import operator
from typing import Optional, Sequence, Any, List
from langchain_core.callbacks import Callbacks, CallbackManagerForRetrieverRun
from langchain_core.documents import BaseDocumentCompressor, Document
from langchain_core.retrievers import BaseRetriever, RetrieverLike
from langchain.retrievers.document_compressors.cross_encoder import BaseCrossEncoder
from langchain.retrievers.document_compressors.base import BaseDocumentCompressor
from pydantic import ConfigDict

# Document compressor that uses CrossEncoder for reranking.
class OriginalCrossEncoderReranker(BaseDocumentCompressor):
    model: BaseCrossEncoder
    top_n: int = 3
    model_config = ConfigDict(
        arbitrary_types_allowed=True,
        extra="forbid",
    )

    def compress_documents(
        self,
        documents: Sequence[Document],
        query: str,
        callbacks: Optional[Callbacks] = None,
    ) -> Sequence[Document]:
        scores = self.model.score([(query, doc.metadata['original']) for doc in documents])
        docs_with_scores = list(zip(documents, scores))
        result = sorted(docs_with_scores, key=operator.itemgetter(1), reverse=True)
        return [doc for doc, _ in result[: self.top_n]]
    

# Retriever that wraps a base retriever and compresses the results.
class OriginalContextualCompressionRetriever(BaseRetriever):
    base_compressor: BaseDocumentCompressor
    base_retriever: RetrieverLike
    model_config = ConfigDict(arbitrary_types_allowed=True,)

    def _get_relevant_documents(
        self,
        query: str,
        *,
        run_manager: CallbackManagerForRetrieverRun,
        **kwargs: Any,
    ) -> List[Document]:
        processed_query = extract_nouns(query)
        docs = self.base_retriever.invoke(
            processed_query, config={"callbacks": run_manager.get_child()}, **kwargs
        )
        if docs:
            compressed_docs = self.base_compressor.compress_documents(
                docs, query, callbacks=run_manager.get_child()
            )
            return list(compressed_docs)
        else:
            return []

In [17]:
# Sparse topk 50 -> Dense top 5
bm25_retriever = BM25Retriever.from_documents(split_docs_nouns)
bm25_retriever.k = 50  # BM25 검색기의 k 설정 (가장 유사한 문서 5개 반환)

model = HuggingFaceCrossEncoder(model_name="dragonkue/bge-reranker-v2-m3-ko")
compressor = OriginalCrossEncoderReranker(model=model, top_n=topk)
compression_retriever = OriginalContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=bm25_retriever
)

In [18]:
# reranker retrieval
rerank_eval = eval_set.copy()
rerank_eval['reference'] = ""
for idx, row in rerank_eval.iterrows():
    query = row['user_message']
    references = [ref.metadata['original'] for ref in compression_retriever.invoke(query)]
    rerank_eval.loc[idx, 'reference'] = str(references)

rerank_eval.head()

,id,paragraph,problems,keyword,user_message,reference
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",{'question': '상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두...,"남인, 기사환국","상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",['조선의 농업 장려 정책 성세창이 아뢰기를 “임금이 나라를 다스리는 데 백성을 교...
1,generation-for-nlp-426,"한양이라는 이름은 조선 왕조가 건국되면서 수도로 삼은 지역의 이름으로, 한강의 북쪽...","{'question': '한양에 대한 설명으로 옳지 않은 것은?', 'choices...","남경, 한양","한양이라는 이름은 조선 왕조가 건국되면서 수도로 삼은 지역의 이름으로, 한강의 북쪽...","['도참사상은 풍수지리(風水地理)와 결합해 나타나는 경우가 많았는데, 고려 시대 묘..."
2,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,"{'question': '(가) 지역에 대한 설명으로 옳은 것은?', 'choice...","서경, 동녕부",나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,"['삼한(三韓)의 여러 나라에서 천신(天神)에게 제사를 드리던 성스러운 장소.', ..."
3,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,"{'question': '밑줄 친 ‘그’에 대한 설명으로 옳은 것은?', 'choi...","김유신, 김춘추",이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,"['군사 관련 업무를 담당한 신라의 관부(官府).', '고구려가 수⋅당의 침략을 막..."
4,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...","{'question': '(가) 인물이 추진한 정책으로 옳지 않은 것은?', 'ch...","흥선대원군, 비변사","선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",['서원은 풍기 군수주세붕이 세운백운동 서원이 시초이다.서원에서는 봄⋅가을로 향음 ...


In [19]:
# 결과 계산 및 출력
rerank_eval_res, hit_at_k_rerank, mrr_at_k_rerank = evaluate_hit_mrr(rerank_eval)

print("[Reranking Retrieval]")
print(f"Hit@{topk}: {hit_at_k_rerank:.4f}")
print(f"MRR@{topk}: {mrr_at_k_rerank:.4f}")

[Reranking Retrieval]
Hit@5: 0.7260
MRR@5: 0.5557


In [20]:
# 결과 하나씩 보기
idx = 0
row = rerank_eval_res.iloc[idx]
references = eval(row['reference'])

print(f"[결과]")
print(f"Hit: {row['hit']}")    # 검색된 문서 중, 연관 문서가 존재하는지 여부 (T/F)
print(f"Rank: {row['rank']}\n")  # 검색된 문서 중, 연관 문서의 순위 (1, ..., TopK)
print(f"[키워드]\n{row['keyword']}\n")
print(f"[문제]\n{row['user_message']}\n")
print("[검색 결과]")
for i, ref in enumerate(references):
    print(f"Result {i+1}: {ref}")

[결과]
Hit: True
Rank: 3

[키워드]
남인, 기사환국

[문제]
상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服) 절차에 대하여 논한 것이 신과는 큰 차이가 있었습니다 . 장자를 위하여 3년을 입는 까닭은 위로 ‘정체(正體)’가 되기 때문이고 또 전 중(傳重: 조상의 제사나 가문의 법통을 전함)하기 때문입니다 . …(중략) … 무엇보다 중요한 것은 할아버지와 아버지의 뒤를 이은 ‘정체’이지, 꼭 첫째이기 때문에 참 최 3년 복을 입는 것은 아닙니다 .”라고 하였다 .－현종실록 －ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 .ㄷ.정조 시기에 탕평 정치의 한 축을 이루었다 .ㄹ.이 이와 성혼의 문인을 중심으로 형성되었다.
상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?
ㄱ, ㄴ ㄱ, ㄷ ㄴ, ㄹ ㄷ, ㄹ

[검색 결과]
Result 1: 조선의 농업 장려 정책 성세창이 아뢰기를 “임금이 나라를 다스리는 데 백성을 교화시키는 것이 중요합니다. 그러나 먼저 살게 한 뒤에 교화시키는 것이 옳습니다. 세종 임금이 농상(農桑)에 적극 힘쓴 까닭에 수령들이 사방을 돌면서 살피고 농상을 권하였으므로 들에 경작하지 않은 땅이 없었습니다. 요즘에는 백성 중에 힘써 농사짓는 사람이 없고, 수령도 들에 나가 농상을 권하지 않습니다. 감사 또한 권하지 않습니다. 특별히 지방에 타일러 농상에 힘쓰도록 함이 어떻습니까?”라고 하였다. 왕이 8도 관찰사 에게 농상을 권하는 글을 내렸다.           〈 중종 실록〉
Result 2: 기해예송 때는 『경국대전(經國大典)』에서 장자와 차자 모두 기년복을 입는다는 규정에 따라 장렬왕후는 1년 동안 상복(喪服)을 입었다. 표면적으로는 서인이 승리하였으나 효종의 지위 문제가 완전히 마무리되지 못한 상황이었기 때문에 언제든지 문제가 다시 불거질 수 있었다. 결국 1674년(현종 15) 효종 비 인선왕후의 죽음으로 또다시 장렬왕후가 상복을 입

---
# Reference 문서 반환
- RAG 실험을 위해, 데이터셋에 retrieve된 문서들 추가 저장

In [23]:
eval_set = pd.read_csv("../data/rag_eval_kor_history.csv")
prompt = "{paragraph}\n{question}\n{choices}"

In [24]:
# retrieval은 추후 성능 잘 나오는걸로 수정 (우선은 bm25로 설정됨)
bm25_retriever = BM25Retriever.from_documents(split_docs_nouns)
bm25_retriever.k = 5  # BM25 검색기의 k 설정 (가장 유사한 문서 5개 반환)

# retrieval 결과 저장
eval_set['reference'] = ""
for idx, row in eval_set.iterrows():
    problems = literal_eval(row['problems'])
    question = problems['question']
    choices = problems['choices']
    choices_str = " ".join(choices)
    query = prompt.format(paragraph=row['paragraph'], question=question, choices=choices_str)
    
    processed_query = extract_nouns(query)  # 검색어 전처리
    results = bm25_retriever.invoke(processed_query)  # BM25 검색 수행
    references = [ref.metadata['original'] for ref in results]
    eval_set.loc[idx, 'reference'] = str(references)

In [25]:
eval_set.head()

,id,paragraph,problems,keyword,reference
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",{'question': '상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두...,"남인, 기사환국",['기해예송 때는 『경국대전(經國大典)』에서 장자와 차자 모두 기년복을 입는다는 규...
1,generation-for-nlp-426,"한양이라는 이름은 조선 왕조가 건국되면서 수도로 삼은 지역의 이름으로, 한강의 북쪽...","{'question': '한양에 대한 설명으로 옳지 않은 것은?', 'choices...","남경, 한양",['조선 후기 시전(市廛)은 도성 상업의 중심지였다. 그러나 18세기 후반 종로에 ...
2,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,"{'question': '(가) 지역에 대한 설명으로 옳은 것은?', 'choice...","서경, 동녕부",['몽골 침략으로 소실된 초조대장경을 대신하여고종때에는 대장경을 다시 만들었다. 대...
3,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,"{'question': '밑줄 친 ‘그’에 대한 설명으로 옳은 것은?', 'choi...","김유신, 김춘추","['안원왕(安原王, 재위 531~545)이 죽은 뒤 왕위 계승 분쟁으로 고구려의 내..."
4,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...","{'question': '(가) 인물이 추진한 정책으로 옳지 않은 것은?', 'ch...","흥선대원군, 비변사",['고종의 즉위(1863)로 정치적 실권을 잡은흥선 대원군은 왕조의 위기를 극복하고...


In [26]:
eval_set.to_csv("../data/rag_eval_kor_history_bm25.csv", index=False)